Web Crawling & Scraping (Web Scraping from a Yelp page) - CODING CHALLENGE !!!

Web Scraping using Python

We use BeautifulSoup package in Python - to perform web scraping, whereby
we attempt to extract out only certain text content.

In this lab, we attempt to scrape a Yelp restaurant review page.
We chose the Din Tai Fung restaurant in Raffles City Shopping Centre.
https://www.yelp.com/biz/din-tai-fung-singapore-4?osq=Restaurants

We attemp to extract the restaurant reviews. In particular, we are interested in:
1. Reviewer's name
2. Reviewer's location
3. Review score (star rating)
4. Review comment

Please note that there are many 'pages' of reviews.
If you scroll down to the bottom of the page, you will see that the current page is 1.
And, you can click on "2" to navigate to another page of reviews.
You can click on "3" to navigate to another page of reviews.
And so on...

In this lab exercise, we are only interested in the FIRSTE page and want to web scrape from this 1st page ONLY.
In order to programmatically download reviews from multiple pages, we will have to resort to "web crawling".
You will need to use a package called 'Selenium' for this.


*** CODING CHALLENGE ***
The script below... attempts to retrieve only the 1st review. And some parts of the code may need further debugging.

1. Modify the code to extract out ALL reviews on the 1st page.

2. Output the extracted reviews in this format.
- 1st row of the output file is the header (author_name, author_location, review_rating, review_comment).
- 2nd row onwards must contain actual data.
- The output is TAB delimited.
- You do not have to put double quotation marks around each value.
- Please note that the review comment must be a single/one string value. Make sure to append multiple lines of text into single/one string.

If you can write the entire code in a more efficient and cleaner manner, by all means -
Please do so.

If and when you are done with the coding challenge, please kindly check Slack #troubleshoot channel
and help guide others struggling.

In [54]:
######## Step 1 ########
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [45]:
path = "/Users/JosiahTheGreat/Documents/[IS434] Social Analytics/IS434/Week7/chromedriver"

#     options = Options()
#     # options.headless = True
#     options.add_argument("--incognito")
#     driver = webdriver.Chrome(path, options=options)
######## Step 2 ########
'''
Let's use Selenium to visit Yelp restaurant review webpage
'''
review_webpage_url = "https://www.yelp.com/biz/din-tai-fung-singapore-4?osq=Restaurants"

browser = webdriver.Chrome(path)
browser.get(review_webpage_url)
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

/var/folders/5w/l3l1gt911hbf2wt84hr0t4z80000gp/T/ipykernel_9062/2243964142.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)


In [53]:
######## Step 3 ########
'''
We can now use BeautifulSoup to inspect the HTML
'''
source = browser.page_source
data = bs(source, 'html.parser')

body = data.find('body')

'''
prettify() function will do indentation so that the extracted HTML content has
HTML tags nicely indented for easy viewing.
Try uncommenting the below line and print.
'''
print(body.prettify())

<body class="" style="margin: 0; padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline; line-height: 1;">
 <noscript>
 </noscript>
 <yelp-react-root>
  <div data-hypernova-hydration-status="1" data-hypernova-id="10080036-4305-4eb4-a4c2-f84d6f7d6d04" data-hypernova-key="yelpfrontend__144__yelpfrontend__GondolaBizDetails__dynamic">
   <div class="toast-list__09f24__miHvH border-color--default__09f24__NPAKY">
   </div>
   <span class="">
    <!-- <a href="/biz/outlook-autumn-market-fundamental-catwalk-flimsy-roost-legibility-individualism-grocer-predestination-4">yelp</a> -->
   </span>
   <div class="border-color--default__09f24__NPAKY background-color--white__09f24__ulvSM" data-lcp-target-id="CONSUMER_HEADER">
    <div class="css-fksa8h padding-t1-5__09f24__VNyRg padding-b1-5__09f24__M_AcV border-color--default__09f24__NPAKY background-color--gray-extra-light__09f24__UbDJH">
     <div class="a11y-menu-container__09f24__V3Frf border-color--default__09f24__NPAKY"

In [55]:
######## Step 4 ########
'''
Get reviewer's name
'''
reviews=browser.find_elements(By.XPATH , "//ul[contains(@class,'undefined')]//li[contains(@class,'margin-b5')]//div[contains(@class,'review')]")
values=[review.find_element(By.XPATH,".//div[contains(@class,'user-passport')]//a").text for review in reviews]
print(values)


['Ik S.', 'Christine R.', 'Tram N.', 'Sylvia K.', 'Bob U.', 'Michael B.', 'C C.', 'Lauren N.', 'Anna R.', 'Ernie K.']


In [56]:
######## Step 5 ########
'''
Get reviewer's location
'''
reviews=browser.find_elements(By.XPATH ,"//ul[contains(@class,'undefined')]//li[contains(@class,'margin-b5')]//div[contains(@class,'review')]")
values=[review.find_element(By.XPATH ,".//div[contains(@class,'user-passport')]//div[contains(@class, 'responsive-hidden')]").text for review in reviews]
print(values)


['American Canyon, CA', 'Houston, TX', 'San Jose, CA', 'Seattle, WA', 'Dublin, CA', 'San Diego, CA', 'Jersey City, NJ', 'Fort Worth, TX', 'Kailua, HI', 'Dallas, TX']


In [57]:
######## Step 6 ########
'''
Get review (star) rating
'''
reviews=browser.find_elements(By.XPATH ,"//ul[contains(@class,'undefined')]//li[contains(@class,'margin-b5')]//div[contains(@class,'review')]")
values=[review.find_element(By.XPATH ,".//div[contains(@class,'i-stars')]").get_attribute("aria-label") for review in reviews]
print(values)

['5 star rating', '4 star rating', '3 star rating', '4 star rating', '2 star rating', '5 star rating', '4 star rating', '4 star rating', '5 star rating', '4 star rating']


In [58]:
######## Step 7 ########
'''
Get review text

If there are multiple sentences/paragraphs, append them into a single String variable.
print it so that we can see in the output console.
'''

reviews=browser.find_elements(By.XPATH ,"//ul[contains(@class,'undefined')]//li[contains(@class,'margin-b5')]//div[contains(@class,'review')]")
values=[review.find_element(By.XPATH ,".//span[contains(@class,'raw')]").text.replace("\n"," ").replace("\\'","'") for review in reviews]
string = ""
for sentence in values:
    string+=sentence
print(string)

5th time here!    Busy Busy  Dim Sum was on point.  No surprise there, right?  Hot and Sour soup was the bomb!Always nice to eat at one of my favorite restaurants that serve soup dumpling. If you are craving good Chinese food come here.Overrated.  If you order 1 or 2 orders of dumplings then you might be able to distinguish the taste but we ordered more than 6 orders (all different ones) so I felt that it all tasted the same after my 2nd dumpling.  The oriental salad was delicious and my favorite. The fried rice was incredibly tasty, too. The duck wrap I ordered was good but I would not order it again. I was not a fan of their BBQ pork buns (all dough and barely any meat filling).  Service was subpar.  We were ignored for a short while.  I'd pick the dim sum at the hawker centers instead.I finally got to try the Din Tai Fung's famous steamed pork dumplings. There's a location in the Raffles City shopping center.  The verdict: pretty tasty! How do they make these? How do they get the so